In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.dirname(sys.path[0]))
from ast import literal_eval

df = pd.read_csv('../data/processed/pipeline/test/ie_hyperion.csv', converters={'Stralci': literal_eval, 'Repertori': literal_eval})

In [2]:
texts = df['Testo'].tolist()

In [4]:
from src.models.nltk_segmenter import NLTKSegmenter

nltk_seg = NLTKSegmenter()

preds = df['Testo'].map(nltk_seg.predict)

[nltk_data] Downloading package punkt to /Users/michele/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
preds.to_list()

[['Comunque, se tutti i giornalisti che oggi si stracciano le vesti per le invettive di Conte a quei due avessero mai fatto correttamente il proprio lavoro, smontando le cazzate di quei due, invece di riportarle paro paro, forse oggi starebbero più tranquilli'],
 ['"Ha suscitato scandalo la proposta di Salvini di riaprire le chiese a Pasqua.',
  'Non altrettanto scandalo aveva suscitato l’idea di Renzi di riaprire le librerie, né quella della Confindustria di tenere aperte le imprese.',
  'Il leader della Lega mescola certo con troppa superficialità il profano della politica con il sacro della preghiera.',
  'E le esigenze di distanziamento sociale rendono evidentemente impossibile ciò che chiede.',
  'Ma le reazioni che ha ricevuto, quasi sdegnate, fanno riflettere.',
  'La paradossale verità è che oggi cultura e industria ci appaiono strumenti di rinascita e riscatto più idonei della religione.',
  'Il processo di secolarizzazione, anche nel Paese più cattolico d’Europa, ha ormai esp